In [2]:
from eComply.eComply import eComply
from ParksGIS.ParksGIS import GISFactory, Server, LayerServerGen, LayerQuery, LayerEdits
from pandas import DataFrame
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
# from Transformer import Transformer

Filters

In [ ]:
from datetime import datetime
from pandas import merge
from typing import Any, Literal


def to_json(obj: Any) -> str:
    return obj.to_json(orient="records", date_format="iso")


def applyEdits(dict: dict) -> dict:
    if dict.get("applyEdits") is not None and dict["applyEdits"].__len__() > 0:
        dict["server"].apply_edits(dict["applyEdits"].values())
    return dict


# def extractChanges(dict: dict) -> dict:
#     changes = dict["server"].extractChanges([dict["LayerServerGen"]])

#     dict = {
#         "applyEdits": {
#             3: LayerEdits(
#                 3,
#                 updates=[
#                     {
#                         "attributes": {
#                             "OBJECTID": 1,
#                             "Contract": changes["layerServerGens"][0]["serverGen"]
#                             // 1000,
#                         }
#                     }
#                 ],
#             )
#         },
#     }

#     objectIds = (
#         changes["edits"][0]["objectIds"]["adds"]
#         + changes["edits"][0]["objectIds"]["updates"]
#     )
#     if 0 == objectIds.__len__():
#         dict["sendEdits"] = DataFrame()
#     else:
#         dict["sendEdits"] = server.query([dict["LayerQuery"].getQuery(objectIds)])

#     print(to_json(dict["sendEdits"][:2]))
#     return dict


def contractExtractChanges(
    repo: Server,
    serverGen: int,
    service: eComply,
) -> dict:
    layerId = 1
    changes = repo.extractChanges([LayerServerGen(layerId, serverGen)])

    dict = {
        "service": service,
        "server": repo,
        "serverGen": changes["layerServerGens"][0]["serverGen"] // 1000,
    }

    objectIds = (
        changes["edits"][0]["objectIds"]["adds"]
        + changes["edits"][0]["objectIds"]["updates"]
    )
    if 0 == objectIds.__len__():
        dict["sendEdits"] = DataFrame()
    else:
        dict["sendEdits"] = repo.query(
            [
                LayerQuery(
                    layerId,
                    ["*"],
                    "OBJECTID IN ("
                    + ",".join(str(i) for i in objectIds)
                    + ") AND EcomplyContract = 1",
                )
            ]
        )[layerId]

    print("Contracts Extracted: " + str(dict["sendEdits"].__len__()))
    # print(to_json(dict["sendEdits"][:1]))
    return dict


def contractSendEdits(dict: dict) -> dict:
    count = dict["sendEdits"].__len__()
    if 0 < count:
        print(to_json(dict["sendEdits"][:1]))
        dict["service"].postContracts(to_json(dict["sendEdits"][:1]))
    print("Contracts Pushed: " + str(count))
    return dict


def workOrderExtractChanges(
    repo: Server,
    serverGen: int,
    service: eComply,
) -> dict:
    layerId = 0
    changes = repo.extractChanges([LayerServerGen(layerId, serverGen)])

    dict = {
        "service": service,
        "server": repo,
        "serverGen": changes["layerServerGens"][0]["serverGen"] // 1000,
    }

    objectIds = (
        changes["edits"][0]["objectIds"]["adds"]
        + changes["edits"][0]["objectIds"]["updates"]
    )
    if 0 == objectIds.__len__():
        dict["sendEdits"] = DataFrame()
    else:
        dict["sendEdits"] = repo.query(
            [
                LayerQuery(
                    layerId,
                    [
                        "InspectionGlobalID",
                        "Type",
                        "Status",
                        "LocationDetails",
                        "ActualFinishDate",
                        "Comments",
                        "Contract",
                        "CancelReason",
                        "GlobalID",
                        "ClosedDate",
                        "ClosedByERN",
                        "ClosedByName",
                        "CancelDate",
                        "CancelByERN",
                        "CancelByName",
                        "CreatedDate",
                        "CreatedBYERN",
                        "CreatedByName",
                        "UpdatedDate",
                        "UpdatedByERN",
                        "UpdatedByName",
                        "WOEntity",
                        "PROJSTARTDATE",
                        "Project",
                        "RecommendedSpecies",
                        "ClosedBySystem",
                        "OBJECTID",
                    ],
                    "OBJECTID IN (" + ",".join(str(i) for i in objectIds) + ")",
                )
            ]
        )[layerId].rename(
            columns={
                "LocationDetails": "Location",
                "GlobalID": "WorkOrderGlobalId",
                "PROJSTARTDATE": "ProjStartDate",
                "RecommendedSpecies": "RecSpecies",
                "OBJECTID": "ObjectId",
            }
        )

    print("Workorders Extracted: " + str(dict["sendEdits"].__len__()))
    # print(to_json(dict["sendEdits"][:1]))
    return dict


def hydrateInspection(dict: dict) -> dict:
    edits = dict["sendEdits"]
    ids = edits[~edits["InspectionGlobalID"].isna()]["InspectionGlobalID"]
    if 0 == ids.__len__():
        return dict

    layerId = 4
    inspections = (
        dict["server"]
        .query(
            [
                LayerQuery(
                    layerId,
                    ["PlantingSpaceGlobalID", "GlobalID"],
                    "GlobalID IN ('" + "','".join(str(id) for id in ids) + "')",
                )
            ]
        )[layerId]
        .rename(columns={"GlobalID": "InspectionGlobalID"})
    )
    print("Inspections found " + str(inspections.__len__()))

    edits = dict["sendEdits"] = merge(
        edits, inspections, on="InspectionGlobalID", how="left"
    )
    del edits["InspectionGlobalID"]

    return dict


def hydratePlantingSpace(dict: dict) -> dict:
    edits = dict["sendEdits"]
    ids = edits[~edits["PlantingSpaceGlobalID"].isna()]["PlantingSpaceGlobalID"]
    if 0 == ids.__len__():
        return dict

    layerId = 2
    plantingSpaces = (
        dict["server"]
        .query(
            [
                LayerQuery(
                    layerId,
                    [
                        "ParkName",
                        "ParkZone",
                        "Borough",
                        "CommunityBoard",
                        "BuildingNumber",
                        "StreetName",
                        "CityCouncil",
                        "StateAssembly",
                        "GISPROPNUM",
                        "CrossStreet1",
                        "CrossStreet2",
                        "PlantingSpaceOnStreet",
                        "ObjectID",
                        "GlobalID",
                    ],
                    "GlobalID IN ('" + "','".join(str(id) for id in ids) + "')",
                )
            ]
        )[layerId]
        .rename(
            columns={
                "GlobalID": "PlantingSpaceGlobalID",
                "OBJECTID": "PlantingSpaceId",
                "PlantingSpaceOnStreet": "OnStreetSite",
            }
        )
    )
    print("Planting Spaces found " + str(plantingSpaces.__len__()))

    dict["sendEdits"] = merge(
        edits, plantingSpaces, on="PlantingSpaceGlobalID", how="left"
    )

    return dict


def workOrderSendEdits(dict: dict) -> dict:
    count = dict["sendEdits"].__len__()
    if 0 < count:
        print(to_json(dict["sendEdits"][:1]))
        dict["service"].postWorkOrders(to_json(dict["sendEdits"][:1]))
    print("Work Orders Pushed: " + str(dict["sendEdits"].__len__()))
    return dict


def workOrderGetChanges(
    service: eComply,
    repo: Server,
    serverGen: int,
) -> dict:
    fromDateTime = datetime.fromtimestamp(serverGen)
    edits = service.getWorkOrders(fromDateTime)
    return {
        "applyEdits": {0: LayerEdits(0, updates=edits)},
        "server": repo,
    }


def update_associated_inspection_HasActiveWorkOrder(dict: dict) -> dict:
    edits = dict["applyEdits"][0].updates
    ids = edits[~edits["plantingSpaceGlobalId"].isna()]["plantingSpaceGlobalId"]
    if 0 == ids.__len__():
        return dict

    layerId = 4
    inspections = (
        dict["server"]
        .query(
            [
                LayerQuery(
                    layerId,
                    [
                        "PlantingSpaceGlobalID",
                        "HasActiveWorkOrder",
                    ],
                    "PlantingSpacGlobalID IN ('"
                    + "','".join(str(id) for id in ids)
                    + "')",
                )
            ]
        )[layerId]
        .rename(
            columns={
                "PlantingSpaceGlobalID": "plantingSpaceGlobalId",
            }
        )
    )

    edits = merge(inspections, edits, on="plantingSpaceGlobalId", how="left")
    edits.loc[
        edits["Status"] == Literal["Closed", "Canceled"], "HasActiveWorkOrder"
    ] = 0

    dict["applyEdits"][layerId](
        LayerEdits(
            layerId,
            updates=edits[
                "OBJECTID",
                "HasActiveWorkOrder",
            ].to_json(orient="records"),
        ),
    )
    return dict


def update_associated_platingSpace_Address(dict: dict) -> dict:
    edits = dict["applyEdits"][0].updates
    ids = edits[~edits["plantingSpaceGlobalId"].isna()]["plantingSpaceGlobalId"]
    if 0 == ids.__len__():
        return dict

    layerId = 2
    plantingSpaces = (
        dict["server"]
        .query(
            [
                LayerQuery(
                    layerId,
                    [
                        "GlobalID",
                        "BuildingNumber",
                        "StreetName",
                        "CrossStreet1",
                        "CrossStreet2",
                    ],
                    "GlobalID IN ('" + ",".join(str(id) for id in ids) + "')",
                )
            ]
        )[layerId]
        .rename(
            columns={
                "GlobalID": "plantingSpaceGlobalId",
            }
        )
    )

    # Transformer.update(
    #     plantingSpaces,
    #     edits,
    #     "plantingSpaceGlobalId",
    #     {
    #         "BuildingNumber": {"Source": "BuildingNumber"},
    #         "StreetName": {"Source": "StreetName"},
    #         "CrossStreet1": {"Source": "CrossStreet1"},
    #         "CrossStreet2": {"Source": "CrossStreet2"},
    #     },
    # )

    dict["applyEdits"][layerId](
        LayerEdits(
            layerId,
            updates=plantingSpaces.to_json(orient="records"),
        ),
    )
    return dict


def contractGetChanges(
    service: eComply,
    repo: Server,
    serverGen: int,
) -> dict:
    fromDateTime = datetime.fromtimestamp(serverGen)
    edits = service.getContracts(fromDateTime)
    return {
        "applyEdits": {
            1: LayerEdits(1, updates=edits),
        },
        "server": repo,
    }


def workOrderLineItemGetChanges(
    service: eComply,
    repo: Server,
    serverGen: int,
) -> dict:
    fromDateTime = datetime.fromtimestamp(serverGen)
    edits = service.getWorkOrderLineItems(fromDateTime)
    return {
        "applyEdits": {
            2: LayerEdits(2, updates=edits),
        },
        "server": repo,
    }

Pipeline


In [ ]:
proxy = "@bcpxy.nycnet:8080"

import os

# Set proxy incase environment not set
os.environ["HTTP_PROXY"] = proxy
os.environ["HTTPS_PROXY"] = proxy
# bypass proxy on parks domains
os.environ["NO_PROXY"] = ".parks.nycnet"

arcGIS = "https://formsgisportal.parks.nycnet"
# arcGIS = 'https://stg-formsgisportal.parks.nycnet'
# arcGIS = 'https://dev-formsgisportal.parks.nycnet'

factory = GISFactory(
    url=arcGIS + "/portal/home",
    username="forms.python_user",
    password="formsPython24*",
    verify_cert=False
)
# eComplyRepo = factory.CreateFeature(arcGIS + '/server/rest/services/eComply/eComplyContract/FeatureServer')
dataPushRepo = factory.CreateFeature(
    arcGIS + "/server/rest/services/DataPush/ForMSDataPush/FeatureServer"
)

service = eComply(
    baseURL="https://nycparks-stage.ecomply.us/WebAPI",
    username="ff@ecomply.us",
    password="!test123",
)

# Push DomainValues
# dataPushRepo = factory.CreateFeature(arcGIS + 'server/rest/services/DataPush/ForMSDataPush/FeatureServer')
# domainValues = dataPushRepo.queryDomains([LayerDomainNames(0, ['WOContract','WOProject','WOEntity','WOStatus','WOCategory','WOType','GenusSpecies'])])
# domains = [
#     {
#         'domainName': domain['name'],
#         'code': str(value['code']),
#         'value': value['name'],
#     }
#     for domain in domainValues for value in domain['codedValues']
# ]
# service.postDomainValues(domains)
# print('DomainValues Pushed')

# serverGens = eComplyRepo.query([LayerQuery(3, ["*"])])[3]
import time

current_time = time.time()
seconds = 24 * 60 * 60 * 2
epoch = current_time - seconds
serverGens = DataFrame({"Contract": [epoch], "WorkOrder": [epoch]})
newServerGens = serverGens.copy()
print(newServerGens)

from toolz import pipe

# sendContracts = pipe(
#     contractExtractChanges(eComplyRepo, serverGens.at[0, 'Contract'], service),
#     contractSendEdits,
# )
# newServerGens['Contract'] = sendContracts['serverGen']
# print(newServerGens)
# eComplyRepo.apply_edits([LayerEdits(3, updates=newServerGens.to_json(orient="records"))])

sendWorkOrders = pipe(
    workOrderExtractChanges(dataPushRepo, serverGens.at[0, "WorkOrder"], service),
    #     hydrateInspection,
    #     hydratePlantingSpace,
    #     workOrderSendEdits,
)
# newServerGens['WorkOrder'] = sendWorkOrders['serverGen']
# print(newServerGens)
# eComplyRepo.apply_edits([LayerEdits(3, updates=newServerGens.to_json(orient="records"))])

# applyWorkOrders = pipe(
#     workOrderGetChanges(service, dataPushRepo, serverGens.at[0, 'WorkOrder']),
#     update_associated_inspection_HasActiveWorkOrder,
#     update_associated_platingSpace_Address,
#     applyEdits
# )

# applyContracts = pipe(
#     contractGetChanges(service, eComplyRepo, serverGens.at[0, 'Contract']),
#     applyEdits
# )

# applyWorkOrderLineItems = pipe(
#     workOrderLineItemGetChanges(service, dataPushRepo, serverGens.at[0, 'WorkOrder']),
#     applyEdits
# )